# FoXS: fitting structures against SAXS data

## Introduction

Nup133 is one of the proteins that make up the yeast
[Nuclear Pore Complex (NPC)](https://salilab.org/npc/). The structure of
this protein has been characterized by both X-ray crystallography and
small angle X-ray (SAXS).

IMP includes a command line tool <tt>foxs</tt> for handling SAXS profiles,
which will be demonstrated in this example.
There is also a [web interface](https://salilab.org/foxs/) available,
which functions similarly.

For full help on the `foxs` tool, run from a command line:

In [1]:
foxs -h

foxs -h 
Usage: <pdb_file1> <pdb_file2> ... <profile_file1> <profile_file2> ... :

Options:
  --help                             Any number of input PDBs and profiles is 
                                     supported. Each PDB will be fitted against
                                     each profile.
  --version                          FoXS (IMP applications)
                                     Copyright 2007-2019 IMP Inventors.
                                     All rights reserved. 
                                     License: GNU LGPL version 2.1 or later
                                     <http://gnu.org/licenses/lgpl.html>.
                                     Written by Dina Schneidman.
  -s [ --profile_size ] arg (=500)   number of points in the profile
  -q [ --max_q ] arg (=0.50)         max q value
  --min_c1 arg (=0.99)               min c1 value
  --max_c1 arg (=1.05)               max c1 value
  --min_c2 arg (=-2.00)              min c2 value
  --max_c2 arg (=4.00) 

# Calculation

The structure of the C-terminal domain of yeast Nup133 is available in the
[RCSB Protein Data Bank (PDB)](http://www.pdb.org) as code
[3kfo](http://www.pdb.org/pdb/explore/explore.do?structureId=3KFO)
(file `3KFO.pdb`), while the experimental SAXS profile is given in the
`23922_merge.dat` file. The atomic structure can be fit against the
SAXS profile by running FoXS:

In [2]:
foxs 3KFO.pdb 23922_merge.dat

foxs 3KFO.pdb 23922_merge.dat 
begin read_pdb:
  WARNING  Sorry, unable to read atoms from PDB file. Thanks for the effort.
end read_pdb
Computing profile for 3KFO.pdb 1669 atoms 
WARNING  Nitrogen atom not found, using default N form factor for "HET: N  " "MSE"
WARNING  Carbon atom not found, using default C form factor for "HET: CA " "MSE"
WARNING  Carbon atom not found, using default C form factor for "HET: C  " "MSE"
WARNING  Oxygen atom not found, using default O form factor for "HET: O  " "MSE"
WARNING  Carbon atom not found, using default C form factor for "HET: CB " "MSE"
WARNING  Carbon atom not found, using default C form factor for "HET: CG " "MSE"
WARNING  Carbon atom not found, using default C form factor for "HET: CE " "MSE"
3KFO.pdb 23922_merge.dat Chi^2 = 8.76148 c1 = 1.02509 c2 = 3.3952 default chi^2 = 97.6038


FoXS calculates the theoretical profile of the 3KFO structure and fits it
against the experimental profile, and determines the quality of the fit, Chi,
with smaller values corresponding to closer fits. It also generates two
`.dat` files, one containing the theoretical profile and the other
this profile fit against the experimental profile. These files can be input to
a plotting package (two [gnuplot](http://www.gnuplot.info/) input
files with a `.plt` extension are provided).

The fit in this example is not a good one (Chi=2.96). To understand why this
is so, the header of the 3KFO PDB file can be examined in a text editor.
This reveals two problems. Several residues at the N and C termini were not
resolved in the X-ray experiment (9 in total, 2 at the N terminus and 7 at the
C terminus), and the sidechains of 16 other residues could also not be located
(REMARK 465 and REMARK 470 lines). This problem can be resolved by adding in
the missing atoms and residues by using
[MODELLER](https://salilab.org/modeller/), which generated the
<tt>3KFO-fill.B99990005.pdb</tt> file.

\note See `add-missing-residues.py` for the MODELLER script used. It simply
      builds a model of the full sequence of 3KFO using the partial sequence
      as a template (see the alignment file `alignment.ali`). Multiple models
      are generated; each is fit against the SAXS data using FoXS and the
      best-fitting model (`3KFO-fill.B99990005.pdb`) selected.

The fits of the original PDB and the new structure can now be generated by
running:

In [3]:
foxs -g 3KFO.pdb 3KFO-fill.B99990005.pdb 23922_merge.dat

foxs -g 3KFO.pdb 3KFO-fill.B99990005.pdb 23922_merge.dat 
begin read_pdb:
  WARNING  Sorry, unable to read atoms from PDB file. Thanks for the effort.
end read_pdb
Computing profile for 3KFO.pdb 1669 atoms 
WARNING  Nitrogen atom not found, using default N form factor for "HET: N  " "MSE"
WARNING  Carbon atom not found, using default C form factor for "HET: CA " "MSE"
WARNING  Carbon atom not found, using default C form factor for "HET: C  " "MSE"
WARNING  Oxygen atom not found, using default O form factor for "HET: O  " "MSE"
WARNING  Carbon atom not found, using default C form factor for "HET: CB " "MSE"
WARNING  Carbon atom not found, using default C form factor for "HET: CG " "MSE"
WARNING  Carbon atom not found, using default C form factor for "HET: CE " "MSE"
3KFO.pdb 23922_merge.dat Chi^2 = 8.76148 c1 = 1.02509 c2 = 3.3952 default chi^2 = 97.6038
Computing profile for 3KFO-fill.B99990005.pdb 1817 atoms 
3KFO-fill.B99990005.pdb 23922_merge.dat Chi^2 = 1.31118 c1 = 1.02835 c2 = 0.

The output shows that the new structure has a significantly better fit
(Chi=1.15). FoXS generates a gnuplot input file `fit.plt` which will
plot the fits of all of the PDBs given against the profile. This can be
run with:

In [4]:
gnuplot fit.plt

This generates an image, `fit.png`, showing the theoretical profiles
(lines in top panel), the experimental profile (points in top panel) and the
ratios of the experimental to theoretical profiles (bottom panel):

![title](fit.png)